# Communication of each module

## Initializing environment

In [1]:
library(Seurat)
library(CellChat)
library(tidyverse)
library(circlize)
library(ggplotify)
library(RColorBrewer)
library(patchwork)
library(lattice)
library(ComplexHeatmap)
library(data.table)

Attaching SeuratObject

载入需要的程辑包：dplyr


载入程辑包：‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


载入需要的程辑包：igraph


载入程辑包：‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


载入需要的程辑包：ggplot2

── Attaching packages ─────────────────────────────── tidyverse 1.3.2 ──
✔ tibble  3.1.8     ✔ purrr   0.3.5
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────── tidyverse_conflicts() ──
✖ tibble::as_data_frame()  masks igraph::as_data_frame(), dplyr::as_data_frame()
✖ Biobase::combine()       masks BiocGenerics::combine(), dplyr::combine()
✖ purrr::compose()         masks igraph::compose()
✖ tidyr::crossing() 

In [2]:
source("~/snippets/seurat2cc.R")

db <- CellChatDB.human

In [ ]:
whole_mod <- readRDS("./data/whole_mod.rds")

## Create communication objects

In [ ]:
mod_lst <- c("Mod1", "Mod2", "Mod3")

In [ ]:
for (mod in mod_lst){
    cat(sprintf("Processing module %s...\n", mod))
    mod_lst <-  SplitObject(
        subset(whole_mod, subset = module == mod),
        split.by = "group"
    ) %>% lapply(., \(x){
        x@meta.data <- droplevels(x@meta.data)
        cc <- seurat2cc(x, group = "desc", db = db)
    })
    saveRDS(mod_lst, sprintf("./data/comObjs/%sLst.rds", mod))
}

## Regular analysis

In [48]:
mod_name <- "Mod2"

In [49]:
cc_lst <- readRDS(sprintf("./data/comObjs/%sLst.rds", mod_name))
cc <- mergeCellChat(cc_lst, add.names = c("LC", "OC", "OD"))

Merge the following slots: 'data.signaling','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.



### Comparison of communication number/strength

In [34]:
cmp1 <- compareInteractions(cc, group = c("LC", "OC", "OD"))$data %>%
    mutate(meas = "Counts")
cmp2 <- compareInteractions(cc, group = c("LC", "OC", "OD"),
                            measure = "weight")$data  %>%
    mutate(meas = "Weight")
svg(sprintf("./plots/com/%s_cmp.svg", mod_name)
    , width = 8, height = 6)
with(
    rbind(cmp1, cmp2),
    barchart(
        count ~ group | meas,
        scale = list(y = "free"),
        ylab = NULL,
        col = "#5b9bd5"
    )
)
dev.off()

png 
  2

### Communication number

In [35]:
comp_lst <- c("LC", "OC", "OD")

In [36]:
max_weight <- getMaxWeight(cc_lst, attribute = c("idents", "count"))
svg(sprintf("./plots/com/%s_net.svg", mod_name),
    width = 24, height = 6)
par(mfrow = c(1, 3), xpd = TRUE)
for (i in 1:3){
    cat(names(cc_lst[[i]]))
    netVisual_circle(
        cc_lst[[i]]@net$count,
        weight.scale = TRUE,
        label.edge = FALSE,
        edge.weight.max = max_weight[2],
        edge.width.max = 12,
        title.name = sprintf("Number of interactions: %s",
                             comp_lst[[i]])
    )
}
dev.off()

png 
  2

### Comparison of overall infoflow

In [37]:
infoflow <- rankNet(cc, mode = "comparison", comparison = 1:3, stacked = T)$data
setDT(infoflow)
infoflow <- mutate(infoflow,
                   group = factor(group, levels = c("LC", "OC", "OD")))

The text on the y-axis will not be colored for the number of compared datasets larger than 3!



In [38]:
library(RColorBrewer)
col_vec <- rev(brewer.pal(2, "Set1"))

In [39]:
svg(sprintf("./plots/com/%s_infoflow.svg", mod_name),
    width = 6, height = 12)
with(
    infoflow[contribution.scaled > 0,.(
        contribution.scaled, group,
        name = reorder(name, contribution.scaled)
    )],
    barchart(
        name ~ contribution.scaled,
        groups = group, stack = TRUE,
        par.settings = list(
            superpose.polygon = list(col = col_vec)
        ),
        auto.key = list(columns = 3),
        xlab = "Information flow"
    )
)
dev.off()

png 
  2

In [40]:
infoflow_pct <- dcast(infoflow[contribution.scaled > 0, .(
        group, contrib_rel = contribution.scaled /
        sum(contribution.scaled), contribution.scaled
    ), by = c("name")][,.(
        name = fct_reorder(name, contribution.scaled),
        contrib_rel, group)], name ~ group, value.var = "contrib_rel")
infoflow_pct <- data.frame(
    row.names = infoflow_pct$name,
    infoflow_pct[,.(LC = ifelse(is.na(LC), 0, LC),
                    OC = ifelse(is.na(OC), 0, OC),
                    OD = ifelse(is.na(OD), 0 ,OD))])
svg(sprintf("./plots/com/%s_infoflow_hm.svg", mod_name), width = 8, height = 8)
Heatmap(infoflow_pct, col = colorRamp2(
            seq(0, 1, length = 8),
            brewer.pal(8, "Blues")),
        column_order = c("LC", "OC", "OD"),
        column_title = sprintf("Relative information flow of %s", mod_name),
        name = "Percentage"
)
dev.off()

png 
  2

### Identify major sources and targets

In [50]:
num_links <- sapply(cc_lst, \(x){
    rowSums(x@net$count) + colSums(x@net$count) - diag(x@net$count)
})
weight_minmax <- c(min(num_links), max(num_links))
gg <- list()
for (i in 1:3) {
    gg[[i]] <- netAnalysis_signalingRole_scatter(
        cc_lst[[i]], title = comp_lst[i], weight.MinMax = weight_minmax
    )
}
ggsave(
    sprintf("./plots/com/%s_sigrole.svg", mod_name),
    wrap_plots(plots = gg),
    width = 12, height = 6)

Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways

Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways

Signaling role analysis on the aggregated cell-cell communication network from all signaling pathways

